<a href="https://colab.research.google.com/github/mihirsh73/Gender-Recognition/blob/master/ccode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
pip install -U numpy

     |████████████████████████████████| 20.2MB 227kB/s 
ERROR: datascience 0.10.6 has requirement folium==0.2.1, but you'll have folium 0.8.3 which is incompatible.
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: numpy 1.17.5
    Uninstalling numpy-1.17.5:
      Successfully uninstalled numpy-1.17.5


In [0]:
pip install -U scikit-multiflow

     |████████████████████████████████| 16.3MB 309kB/s 


In [0]:
%matplotlib notebook

In [0]:
import skmultiflow

In [0]:
from skmultiflow.trees.hoeffding_tree import HoeffdingTree
from skmultiflow.bayes import NaiveBayes
from skmultiflow.bayes import do_naive_bayes_prediction
from skmultiflow.drift_detection.adwin import ADWIN
from abc import ABCMeta, abstractmethod
from skmultiflow.utils.utils import *
import math
import random
import numpy as np


class HoeffdingAdaptiveTree(HoeffdingTree):


    class NewNode(metaclass= ABCMeta):
       
        def numberLeaves(self):
            pass

        @abstractmethod
        def getErrorEstimation(self):
            pass

        @abstractmethod
        def getErrorWidth(self):
            pass

        @abstractmethod
        def isNullError(self):
            pass

        @abstractmethod
        def killTreeChilds(self, ht):
            pass

        @abstractmethod
        def learnFromInstance(self, X,y, weight, ht,parent, parentBranch):
            pass

        @abstractmethod
        def filterInstanceToLeaves(self, X, myparent, parentBranch,updateSplitterCounts,foundNodes):
            pass


    def __init__(self,*args, **kwarg):
        """HoeffdingTree class constructor."""
        super(HoeffdingAdaptiveTree,self).__init__(*args, **kwarg)
        self.treeRoot = None
        self._pruned_alternate_trees = 0
        self._switch_alternate_trees = 0
        self.alternateTree=0
        

    class AdaSplitNode(HoeffdingTree.SplitNode,NewNode):
       


        def __init__(self, split_test, class_observations, size):
            """SplitNode class constructor"""
            HoeffdingTree.SplitNode.__init__(self, split_test,class_observations,size)
            self.numLeaves = 0
            self.estimationErrorWeight = ADWIN()
            self.ErrorChange = False
            self.alternateTree = None
            self.randomSeed = 1
            self.classifierRandom = random.seed(self.randomSeed)

        def numberLeaves(self):
            
            num_of_leaves = 0
            for child in self._children:
                if child is not None:
                    num_of_leaves += child.number_leaves()

            return num_of_leaves

        
        def calc_byte_size_including_subtree(self):
            
            byteSize = self.__sizeof__()

            if self.alternateTree is not None:
                byteSize += self.alternateTree.calc_byte_size_including_subtree()
            if self.estimationErrorWeight is not None:
                byteSize += self.estimationErrorWeight.get_length_estimation()

            for child in self._children:
                    if child is not None:
                        byteSize += child.calc_byte_size_including_subtree()
            return byteSize

        
        def number_leaves(self):
            
            numleaves = 0
            for child in self._children:
                if child is not None:
                    numleaves += child.number_leaves()
            return numleaves

        def getErrorEstimation(self):
            
            
            return self.estimationErrorWeight._estimation

           
        def getErrorWidth(self):
            
            w = 0.0
            if self.isNullError() is False:
                w = self.estimationErrorWeight._width
            return w

        
        def isNullError(self):
            
            return self.estimationErrorWeight is None



        
        def learnFromInstance(self, X, y, weight, ht, parent, parent_branch):
           

            class_prediction = 0
            #k = np.random.poisson(1.0,self.classifierRandom)
            if self.filter_instance_to_leaf(X, parent, parent_branch).node is not None:
                 res=self.filter_instance_to_leaf(X, y, parent, parent_branch).node.get_class_votes(X, ht)

                 #Get the majority vote
                 max=0
                 maxIdx=0
                 for k ,v in res.items():
                     if v>max:
                         maxIdx=k
                         max = 0
                 class_prediction =maxIdx

            bl_correct = (y == class_prediction)

            if self.estimationErrorWeight is None:
                self.estimationErrorWeight = ADWIN()

            old_error = self.get_error_estimation()

            self.estimationErrorWeight.add_element(0.0 if bl_correct == True else 1.0)
            self.error_change =self.estimationErrorWeight.detected_change()
            

            if self.error_change == True and old_error > self.get_error_estimation():
                self.error_change = False

            if self.error_change ==True:
                self.alternateTree = ht.new_learning_node()
                ht.alternateTrees += 1

            
            elif self.alternateTree is not None and self.alternateTree.is_null_error() == False:
                if self.get_error_width() > 300 and self.alternateTree.get_error_width() > 300:
                    old_error_rate = self.get_error_estimation()
                    alt_error_rate = self.alternateTree.get_error_estimation()
                    fDelta = 0.05
                    fn = 1.0 / (self.alternateTree.get_error_width()) + 1.0 / (self.get_error_width())

                    bound = 1.0 / math.sqrt(2.0 * old_error_rate * (1.0 - old_error_rate) * math.log(2.0 / fDelta)*fn)

                    if bound < old_error_rate - alt_error_rate:
                        ht._active_leaf_node_cnt -= self.numberLeaves()
                        ht._active_leaf_node_cnt += self.alternateTree.numberLeaves()
                        self.killTreeChilds(ht)

                        if parent is not None:
                            parent.setChild(parent_branch, self.alternateTree)
                        else:
                            ht.treeRoot = ht.treeRoot.alternateTree
                        ht._switchAlternateTrees += 1

                    elif bound < alt_error_rate - old_error_rate:
                        if isinstance(self.alternateTree, HoeffdingTree.ActiveLearningNode):
                            self.alternateTree = None
                            self._active_leaf_node_cnt -= 1

                        elif isinstance(self.alternateTree, HoeffdingTree.InactiveLearningNode):#tricky
                            self.alternateTree = None
                            self._inactive_leaf_node_cnt -= 1

                        else:
                            self.alternateTree.killTreeChilds(ht)
                        ht._prunedalternateTree += 1

            if self.alternateTree is not None:
                self.alternateTree.learnFromInstance(X, y, weight, ht, parent, parent_branch)

            child_branch = self.instance_child_index(X)
            child = self.get_child(child_branch)

            if child is not None:
                child.learnFromInstance(X, y, weight, ht, parent, parent_branch) #tricky

        
        def killTreeChilds(self, ht):
            
			
            for child in self._children:
                if child is not None:

                    if isinstance(child, HoeffdingAdaptiveTree.AdaSplitNode):
                        child.killTreeChilds(ht)

                    if isinstance(child, HoeffdingAdaptiveTree.ActiveLearningNode):
                        child = None
                        ht._active_leaf_node_cnt -= 1


                    elif isinstance(child, HoeffdingTree.InactiveLearningNode):
                        child = None
                        ht._inactive_leaf_node_cnt -= 1
                        

        def filterInstanceToLeaves(self, X, myparent, parentBranch,updateSplitterCounts,foundNodes=None):
            


            if foundNodes is None:
               foundNodes = []
            child_index = self.instance_child_index(X)

            if child_index >= 0:
                child = self.get_child(child_index)

                if child is not None:
                    child.filterInstanceToLeaves(X, myparent,parentBranch, updateSplitterCounts,foundNodes)
                else:
                    foundNodes.append(HoeffdingTree.FoundNode(None, self, child_index))

            if self.alteranteTree is not None:
                 self.alternateTree.filterInstanceToLeaves(X, self,  -999, updateSplitterCounts,foundNodes)

            
    def _new_learning_node(self, initial_class_observations=None):
       
        return self.AdaLearningNode(initial_class_observations)


    def new_split_node(self, split_test, class_observations, size):

        
        

        return self.AdaSplitNode(split_test, class_observations, size)


    class AdaLearningNode(HoeffdingTree.LearningNodeNBAdaptive, NewNode):

            
            def __init__(self, initialClassObservations):
                HoeffdingTree.LearningNodeNBAdaptive.__init__(self, initialClassObservations)
                self.estimationErrorWeight= ADWIN()
                self.ErrorChange = False
                self.randomSeed = 1
                self.classifierRandom = random.seed(self.randomSeed)

           
            def calcByteSize(self):
               
                byteSize = self.__sizeof__()
                if self.estimationErrorWeight is not None:
                    byteSize += self.estimationErrorWeight.get_length_estimation()
                return byteSize

         
            def numberLeaves(self):
                """ Return 1"""
                return 1

            
            def getErrorEstimation(self):
               
                if self.estimationErrorWeight is not None:
                    return self.estimationErrorWeight._estimation
                else:
                    return 0

           
            def getErrorWidth(self):
               
                return self.estimationErrorWeight._width

           
            def isNullError(self):
                
                return self.estimationErrorWeight is None

          
            def killTreeChilds(self, ht):
                
                pass

           
            def learnFromInstance(self, X, y, weight, ht, parent, parentBranch):
               
                ClassPrediction = 0

                k = np.random.poisson(1.0, self.classifierRandom)

                if (k > 0):
                    weight = weight * k

                vote = self.get_class_votes(X, ht)

                # Get the majority vote
                max = 0
                maxIdx = 0
                for k, v in vote.items():
                     if v > max:
                         maxIdx = k
                         max=v
                ClassPrediction = maxIdx


                blCorrect = (y == ClassPrediction)

                if (self.estimationErrorWeight is None):
                    self.estimationErrorWeight = ADWIN()

                oldError = self.getErrorEstimation()

                self.estimationErrorWeight.add_element(0.0 if blCorrect == True else 1.0)
                self.ErrorChange = self.estimationErrorWeight.detected_change()


                if self.ErrorChange == True and oldError > self.getErrorEstimation():
                    self.ErrorChange = False

                super().learn_from_instance(X, y, weight, ht)

                weight_seen = self.get_weight_seen()

                if weight_seen - self.get_weight_seen_at_last_split_evaluation() >= ht.grace_period:
                    ht._attempt_to_split(self, parent, parentBranch)
                    self.set_weight_seen_at_last_split_evaluation(weight_seen)

           
            
            def normalize(self,sum, dist):
                
                if sum == 0 and math.isnan(sum):
                    for key, value in dist.items(): 
                        value = value / sum


            
            def get_class_votes(self, X, ht):
             
                
                dist = {}
                
                if (self._mc_correct_weight > self._nb_correct_weight):
                        dist = self.get_observed_class_distribution()
                else:
                        dist = do_naive_bayes_prediction(X, self._observed_class_distribution, self._attribute_observers)

                distSum = sum(dist.values())
                distSum=0
                for key in dist:
                    distSum+=dist[key]

                ds = distSum * self.getErrorEstimation() * self.getErrorEstimation()
                if ds > 0.0:
                    self.normalize(ds, dist)
                return dist

           
            def filterInstanceToLeaves(self, X, parent, parent_branch,
                                        updateSplitterCounts,foundNodes=None):
                                        
                
                if foundNodes is None:
                    foundNodes=[]
                foundNodes.append(HoeffdingTree.FoundNode(self, parent, parent_branch))

            def new_split_node(split_test, class_observations):
                return HoeffdingAdaptiveTree.AdaSplitNode(split_test, class_observations)

    
    def _new_learning_node(self, initial_class_observations=None):
        """Learn New Nodes"""
        return self.AdaLearningNode(initial_class_observations)

    
    def filterInstanceToLeaves(self, X, split_parent, parent_branch, update_splitter_counts):
        
        nodes = []
        self.treeRoot.filterInstanceToLeaves(X, split_parent, parent_branch, update_splitter_counts, nodes)
        return nodes



    
    


    def partial_fit(self, X, y, classes=None, weight=None):
        
        

        if y is not None:
            if weight is None:
                weight = np.array([1.0])
            row_cnt, _ = get_dimensions(X)
            wrow_cnt, _ = get_dimensions(weight)
            if row_cnt != wrow_cnt:
                weight = [weight[0]] * row_cnt
            for i in range(row_cnt):
                if weight[i] != 0.0:
                    self._partial_fit(X[i], y[i], weight[i])


    def _partial_fit(self, X, y, weight):
       

        if self.treeRoot is None:
            self.treeRoot = self._new_learning_node()
            self._active_leaf_node_cnt = 1
        self.treeRoot.learnFromInstance(X, y, weight, self, None, -1)




    def predict(self, X):
       

        
        r, _ = get_dimensions(X)
        predictions = []
        for i in range(r):
            votes = self.get_votes_for_instance(X[i])
            if votes == {}:
                # Tree is empty, all classes equal, default to zero
                predictions.append(0)
            else:
                predictions.append(max(votes, key=votes.get))
        return predictions
    
    def get_votes_for_instance(self, X):
        if self.treeRoot is not None:
            found_nodes = self.filterInstanceToLeaves(X, None, -1, False)
            result = {}
            predictionPaths = 0

            for found_node in found_nodes:
                if found_node.parent != -999:
                    leaf_node = found_node.node
                    if leaf_node is None:
                        leaf_node = found_node.parent
                    dist = leaf_node.get_class_votes(X, self)
                    result.update(dist)
            return result
        else:
            return {}

In [0]:
from skmultiflow.data import FileStream
from skmultiflow.trees import HoeffdingTree
from skmultiflow.evaluation import EvaluatePrequential

# 1. Create a stream
stream = FileStream("/content/elec.csv")
stream.prepare_for_use()

# 2. Instantiate the HoeffdingTree classifier
ht = HoeffdingTree()

# 3. Setup the evaluator
evaluator = EvaluatePrequential(pretrain_size=1000,
                                max_samples=10000,
                                output_file='results.csv')

# 4. Run evaluation
evaluator.evaluate(stream=stream, model=ht)

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 1000 sample(s).
Evaluating...
 #################### [100%] [3.32s]
Processed samples: 10000
Mean performance:
M0 - Accuracy     : 0.8256
M0 - Kappa        : 0.6405


[HoeffdingTree(binary_split=False, grace_period=200, leaf_prediction='nba',
               max_byte_size=33554432, memory_estimate_period=1000000,
               nb_threshold=0, no_preprune=False, nominal_attributes=None,
               remove_poor_atts=False, split_confidence=1e-07,
               split_criterion='info_gain', stop_mem_management=False,
               tie_threshold=0.05)]

In [0]:
h = HoeffdingAdaptiveTree(ht)

In [0]:
evaluator = EvaluatePrequential(pretrain_size=1000,
                                max_samples=10000,
                                output_file='results.csv')

# 4. Run evaluation
evaluator.evaluate(stream=stream, model=ht)

Prequential Evaluation
Evaluating 1 target(s).
Pre-training on 1000 sample(s).
Evaluating...
 #################### [100%] [3.48s]
Processed samples: 10000
Mean performance:
M0 - Accuracy     : 0.8389
M0 - Kappa        : 0.6698


[HoeffdingTree(binary_split=False, grace_period=200, leaf_prediction='nba',
               max_byte_size=33554432, memory_estimate_period=1000000,
               nb_threshold=0, no_preprune=False, nominal_attributes=None,
               remove_poor_atts=False, split_confidence=1e-07,
               split_criterion='info_gain', stop_mem_management=False,
               tie_threshold=0.05)]